# FeatureLab — login_attempt


This notebook builds **login_attempt** features for an existing run_id, then inspects the resulting feature table.


In [ ]:
from pathlib import Path
import pandas as pd

from inkswarm_detectlab.config import load_config
from inkswarm_detectlab.features import build_login_features_for_run
from inkswarm_detectlab.io.tables import read_auto

cfg = load_config(Path("configs/skynet_smoke.yaml"))
run_id = cfg.run.run_id or "RUN_XXX_0005"  # set to your run id

# Build (idempotent unless --force)
build_login_features_for_run(cfg, run_id=run_id, force=False)

feat_df = read_auto(Path(cfg.paths.runs_dir) / run_id / "features" / "login_attempt" / "features")
feat_df.head()


In [ ]:
# Basic sanity: label prevalence + a few feature columns
label_cols = [c for c in feat_df.columns if c.startswith("label_")]
feat_df[label_cols].mean().sort_values(ascending=False)


In [ ]:
# Show a couple of features
cols = [c for c in feat_df.columns if "__attempt_cnt" in c][:5] + [c for c in feat_df.columns if c.endswith("failure_rate")][:5]
feat_df[cols].describe().T
